# Case 1 說明
只對第一個 user input 執行 RAG

Retrieve flow and related QA

In [147]:
import os
import time
from openai import OpenAI
from tabulate import tabulate
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')

In [148]:
import pandas as pd
import numpy as np
import faiss

csv_file = 'data\embeddingsV3_3072.csv'
df = pd.read_csv(csv_file)
data = df.to_numpy()

index = faiss.IndexFlatL2(data.shape[1])
index.add(data)

In [149]:
def get_similar_qa(input):
    response = client.embeddings.create(input=input, model="text-embedding-3-large", dimensions=3072)
    question = np.array(response.data[0].embedding, ndmin=2)
    k = 3
    distances, indices = index.search(question, k)
    csv_question = 'data\manual.csv'
    qdf = pd.read_csv(csv_question)
    qarray = qdf.to_numpy()
    answer = [(qarray[i], float(dist)) for dist, i in zip(distances[0], indices[0])]
    history = ""
    min_distance = 1
    Qcount = 0
    for i in range(0,k):
        if answer[i][1]<min_distance:
            Qcount+=1
            history = history +str(i+1)+". "+ answer[i][0][0]+"\n"+answer[i][0][1]+"\n"
    return Qcount, history

def get_similar_flow(input):
    pass

In [150]:
similar = get_similar_qa("How do I find out which email I've linked to?")

print(similar[1])

1. I was wondering if it were possible to change the email linked to my account? Thank you for the help.
We can help link your account to the email that you use more often. Please use the email to register a new account and complete the verification process. After that, send us the old and new email address for the account, and we will manually link it for you.We can help migrate your account. Please use a new email address to register a new account and complete the verification process. After that, send us the old and new email address for the account, and we will manually migrate for you. Note that we can only keep the data of one account after connecting. In other words, the data in another account will be erased. Let us know if you have any questions.
2. How do I change my email? 
Currently, changing your email is not supported in the settings. However, you may contact us to migrate your account for you. We can help migrate your account. Please use a new email address to register a

# Instruction

In [ ]:
instructions = [
    "1. You are Flora customer service chat bot.",
    "2. When user ask a question, you must Follow the steps, resolving the problem one step at a time.",
    "3. If the user asks another question before the previous question is solved, deal with the current question first and then go back to the steps of the previous question for continued processing.",
]

instruction = ""
for i in instructions:
    instruction += i

print(instruction)

In [ ]:
assistant = client.beta.assistants.create(
    name="Flora's Customer Service",
    instructions=instruction,
    model="gpt-3.5-turbo-1106"
    # tools = [
    #     {
    #         "type": "function",
    #         "function": {
    #             "name": "get_history_question_and_answer",
    #             "description": "Get 1 to 3 history questions and answers that is similar to current question",
    #             "parameters": {
    #                 "type": "object",
    #                 "properties": {
    #                     "current_question": {
    #                         "type": "string",
    #                         "description": "user's question"
    #                     }
    #                 },
    #                 "required": ["current_question"]
    #             }
    #         }
    #     }
    # ]
)

thread = client.beta.threads.create()

# Flow

In [ ]:
flow = [
    "Follow the steps to answer questions, if another question comes out, deal with the current problem first, then go back to the step of previous issue if user ask again:",
    "Ask the user for their email.",
    "Ask the user for the time when the tree was killed.",
    "The last step, you must respond based on the reference answer of the user's initial question if history questions and reference answers are provided. If not, you must reply only a special mark:'*'."
]

In [ ]:
user_input = "I accidentally killed my tree. Is there any way to restore it?"

similar_qa = get_similar_qa(user_input)

similar_flow = get_similar_flow(user_input)

In [ ]:
# prompt = "Answer the user according to this process: " + get_similar_flow(flow) + get_similar_qa(user_input)

prompt = ""
for i in flow:
    prompt = prompt + i + "\n"

if similar_qa[0] > 0:
    prompt += "\nHere is " + str(similar_qa[0]) + " history questions and answers related to this question according to Flora's database:\n" + similar_qa[1] + "Here is the question:\n"

prompt += user_input
print(prompt)

# 2nd round

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",

    # Change the comments for this part of the code.
    content = prompt
    # content = "123@gmail.com"
    # content = "It's around 4:00 PM on March 28th."
)

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [ ]:
while run.status == "queued" or run.status == "in_progress":
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
print(run.status)
# print(tabulate(run,headers=['run','content'],tablefmt='simple_outline'))

In [ ]:
time.sleep(5)
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order='asc'
)

In [ ]:
for i in messages.data[-4:]:
    print(i.content[0].text.value)
    print("==========")

# Delete Assistant

In [ ]:
# client.beta.assistants.delete(assistant.id)